In [1]:
import os
import numpy as np
import nibabel as nib
import pandas as pd
import mkl
from time import time

In [5]:
n_workers = 1

In [6]:
# Make sure only maximum n_workers number of cores are used
os.environ["MKL_THREADING_LAYER"] = "sequential"
mkl.set_num_threads(n_workers)
mkl.set_num_threads(n_workers)

1

# Load data

In [7]:
dir_data = './data'

In [8]:
# Load scheme file
scheme_df = pd.read_pickle(os.path.join(dir_data,'scheme.pkl'))

# Load mask
img = nib.load(os.path.join(dir_data,'mask_slice_55.nii.gz'))
mask = img.get_fdata()

data_fname = 'dwi_DGE_slice_55.nii.gz'
# data_fname = 'dwi_DGEp2s_slice_55.nii.gz'

img = nib.load(os.path.join(dir_data,data_fname))
data = img.get_fdata()

# Bound optimization intervals

In [9]:
nR,nC,nS = mask.shape
one = np.ones((nR,nC,nS))
D_par_low = 1.2e-9*one
D_par_high = 3.4e-9*one
D_perp_low = 0.001e-9*one
D_perp_high = 0.2e-9*one

In [10]:
nR,nC,nS

(140, 140, 1)

# Import methods

In [11]:
import shmodel_varpro as varpro

# With spherical mean

In [12]:
# Spherical Harmonics order to use (max is 12) for each shell (at the moment only the same order for each shell is implemented)
sh_order_list = [12,12]

# Regularization type
use_sph_mean = True
gamma = 25 / 12000
reg_type = 'LB'


# Base name for results
output_name = data_fname[:-7]+'_sh_'+str(sh_order_list[0])+ '_' + reg_type +'_gamma_'+   "{:.{}f}".format( gamma, 5 )

if use_sph_mean:
    output_name = output_name + '_biased'
else:
    output_name = output_name + '_unbiased'

print('base name is')
print(output_name)

base name is
dwi_DGE_slice_55_sh_12_LB_gamma_0.00208_biased


In [11]:
b10_idxs = scheme_df[scheme_df.b == scheme_df.b.unique()[-1]].index
b05_idxs = scheme_df[scheme_df.b == scheme_df.b.unique()[-2]].index

In [12]:
if not use_sph_mean:
    data[...,b10_idxs] = data[...,b10_idxs] - np.mean(data[...,b10_idxs],axis=-1)[...,np.newaxis]
    data[...,b05_idxs] = data[...,b05_idxs] - np.mean(data[...,b05_idxs],axis=-1)[...,np.newaxis]
    
# selecting only the high b-balue shells

b10 = scheme_df.b.unique()[-1]
b5 = scheme_df.b.unique()[-2]
bvals_unique = scheme_df.b.unique()[-2:]
idxs = scheme_df[(scheme_df.b==b5) | (scheme_df.b==b10)].index
bvals = scheme_df.b.values[idxs]
vecs = np.c_[scheme_df.x.values[idxs],scheme_df.y.values[idxs],scheme_df.z.values[idxs]]

data_rcs= data[:,:,:,idxs]

In [14]:


start=time()
dpar,dperp = varpro.fit_tensor_sh_all_data_varpro(data_rcs,
                                        vecs,
                                        bvals,
                                        bvals_unique,
                                        sh_order_list,
                                        n_workers,
                                        mask,
                                        D_par_low = D_par_low,
                                        D_par_high = D_par_high,
                                        D_perp_low = D_perp_low,
                                        D_perp_high = D_perp_high,
                                        constrain_sph_mean=False,
                                        lambda_tik=gamma,
                                        positivity=False,
                                        use_sph_mean=use_sph_mean,
                                        neighboorhood_fit=False,
                                        neighboorhood_radius=1,
                                        neighboorhood_size=1,
                                        reg=reg_type)

elapsed_time = time()-start
print('It took ',str(int(elapsed_time /60. )) + ' minutes')

SH ORDER LIST only works with equal SH order per shell so far e.g [12,12]
Fit...
It took  8 minutesomplete expected time 1 secondss4281 out of 8157 complete expected time 9 minutes4321 out of 8157 complete expected time 7 minutes


In [15]:
tmp = dperp*1e9
outImg=nib.Nifti1Image(tmp,img.affine)
out_name = os.path.join(os.path.join('./',output_name+ '_dperp_L-BFGS-B.nii.gz'))
nib.save(outImg, out_name)

tmp = dpar*1e9
outImg=nib.Nifti1Image(tmp,img.affine)
out_name = os.path.join(os.path.join('./',output_name+ '_dpar_L-BFGS-B.nii.gz'))
nib.save(outImg, out_name)

# Without spherical mean

In [16]:
# Spherical Harmonics order to use (max is 12) for each shell (at the moment only the same order for each shell is implemented)
sh_order_list = [12,12]

# Regularization type
use_sph_mean = False
gamma = 2
reg_type = 'eye' #Tikhonov


# Base name for results
output_name = data_fname[:-7]+'_sh_'+str(sh_order_list[0])+ '_' + reg_type +'_gamma_'+   "{:.{}f}".format( gamma, 5 )

if use_sph_mean:
    output_name = output_name + '_biased'
else:
    output_name = output_name + '_unbiased'

print('base name is')
print(output_name)

base name is
dwi_DGE_slice_55_sh_12_eye_gamma_2.00000_unbiased


In [17]:
b10_idxs = scheme_df[scheme_df.b == scheme_df.b.unique()[-1]].index
b05_idxs = scheme_df[scheme_df.b == scheme_df.b.unique()[-2]].index

In [18]:
if not use_sph_mean:
    # subtract the mean from the data
    data[...,b10_idxs] = data[...,b10_idxs] - np.mean(data[...,b10_idxs],axis=-1)[...,np.newaxis]
    data[...,b05_idxs] = data[...,b05_idxs] - np.mean(data[...,b05_idxs],axis=-1)[...,np.newaxis]
    
# selecting only the high b-balue shells

b10 = scheme_df.b.unique()[-1]
b5 = scheme_df.b.unique()[-2]
bvals_unique = scheme_df.b.unique()[-2:]
idxs = scheme_df[(scheme_df.b==b5) | (scheme_df.b==b10)].index
bvals = scheme_df.b.values[idxs]
vecs = np.c_[scheme_df.x.values[idxs],scheme_df.y.values[idxs],scheme_df.z.values[idxs]]

data_rcs= data[:,:,:,idxs]

In [19]:


start=time()
dpar,dperp = varpro.fit_tensor_sh_all_data_varpro(data_rcs,
                                        vecs,
                                        bvals,
                                        bvals_unique,
                                        sh_order_list,
                                        n_workers,
                                        mask,
                                        D_par_low = D_par_low,
                                        D_par_high = D_par_high,
                                        D_perp_low = D_perp_low,
                                        D_perp_high = D_perp_high,
                                        constrain_sph_mean=False,
                                        lambda_tik=gamma,
                                        positivity=False,
                                        use_sph_mean=use_sph_mean,
                                        neighboorhood_fit=False,
                                        neighboorhood_radius=1,
                                        neighboorhood_size=1,
                                        reg=reg_type)

elapsed_time = time()-start
print('It took ',str(int(elapsed_time /60. )) + ' minutes')

ATTENTION: you should have subtracted the spherical mean from the data before passing it as input
SH ORDER LIST only works with equal SH order per shell so far e.g [12,12]
Fit...
It took  7 minutesomplete expected time 1 secondss


In [20]:
tmp = dperp*1e9
outImg=nib.Nifti1Image(tmp,img.affine)
out_name = os.path.join(os.path.join('./',output_name+ '_dperp_L-BFGS-B.nii.gz'))
nib.save(outImg, out_name)

tmp = dpar*1e9
outImg=nib.Nifti1Image(tmp,img.affine)
out_name = os.path.join(os.path.join('./',output_name+ '_dpar_L-BFGS-B.nii.gz'))
nib.save(outImg, out_name)